In [1]:
import os
import json
import pandas as pd
from typing import List, Dict, Any

# ==============================
# 0. 경로 및 기본 설정
# ==============================

COHORT_PATH = "./cohort/cohort_ver50_only_subject_id.csv"

# MIMIC4-hosp-icu 폴더에 admissions, patients, icustays,
# procedures_icd, prescriptions, labevents_troponin 등이 있다고 가정
HOSP_DIR = "../../data/MIMIC4-hosp-icu"
ICU_DIR = "../../data/MIMIC4-hosp-icu"
ED_DIR = "../../data/mimic-iv-ed"
ECG_DIR = "../../data/mimic-iv-ecg"

OUTPUT_DIR = "./cohort"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 전체 Event Log를 하나로 합친 파일
EVENT_LOG_FULL_PATH = os.path.join(OUTPUT_DIR, "event_log_stemi_all.csv")

# 중간 저장 chunk 파일 이름 규칙
CHUNK_VERSION_START = 140
BATCH_SIZE = 100  # 한 번에 처리할 환자 수

# Troponin 양성 기준 (예: 0.04 이상 양성)
TROP_POS_THRESHOLD = 0.04

# PCI ICD 코드 prefix (예시 – 필요시 조정 가능)
PCI_ICD9_PREFIXES = ["00.66", "36.0"]
PCI_ICD10_PREFIXES = ["027", "929"]

# 항혈소판제 이름 리스트 (대소문자 무시)
ANTI_PLT_DRUGS = ["aspirin", "clopidogrel", "ticagrelor", "prasugrel"]


# ==============================
# 1. 공통 유틸
# ==============================

def _to_datetime(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")
    return df


def _attrs_to_json(attrs: Dict[str, Any]) -> str:
    """attributes dict를 JSON 문자열로 변환."""
    clean = {}
    for k, v in attrs.items():
        if isinstance(v, float) and pd.isna(v):
            clean[k] = None
        else:
            clean[k] = v
    return json.dumps(clean, ensure_ascii=False)


# ==============================
# 2. Cohort 및 원본 테이블 로딩
# ==============================

def load_labevents_troponin() -> pd.DataFrame:
    """
    Troponin 전용 labevents_troponin.csv 로딩.
    - HOSP_DIR 아래 또는 현재 작업 디렉토리에서 탐색.
    """
    candidates = [
        os.path.join(HOSP_DIR, "labevents_troponin.csv"),
        "./labevents_troponin.csv",
    ]
    for p in candidates:
        if os.path.exists(p):
            print(f"[LOAD] labevents_troponin.csv 로딩: {p}")
            df = pd.read_csv(p)
            return df

    raise FileNotFoundError(
        "labevents_troponin.csv 파일을 찾을 수 없습니다.\n"
        f"다음 경로 중 하나에 존재해야 합니다.\n{candidates}"
    )


def load_cohort_first_stemi_admission(path: str) -> pd.DataFrame:
    """
    STEMI 대상 환자 코호트 id 목록 로딩.
    - 입력 파일에는 동일 subject_id에 여러 hadm_id(STEMI 입원들)가 있을 수 있음.
    - admissions.admittime 기준으로 각 subject_id마다 '가장 이른 hadm_id' 하나만 남김.
    필수 컬럼: subject_id, hadm_id
    """
    cohort_raw = pd.read_csv(path)

    if "subject_id" not in cohort_raw.columns:
        raise ValueError(f"cohort 파일에 subject_id 컬럼이 없습니다. 현재 컬럼: {list(cohort_raw.columns)}")
    if "hadm_id" not in cohort_raw.columns:
        raise ValueError(
            "cohort 파일에 hadm_id 컬럼이 없습니다.\n"
            "STEMI 분석은 hadm_id(입원 단위) 기준으로 해야 하므로,\n"
            "cohort_ver50_only_subject_id.csv를 subject_id, hadm_id 두 컬럼을 포함하도록 다시 만들어 주세요."
        )

    cohort_all = cohort_raw[["subject_id", "hadm_id"]].drop_duplicates()

    admissions_path = os.path.join(HOSP_DIR, "admissions.csv")
    admissions = pd.read_csv(admissions_path)[["subject_id", "hadm_id", "admittime"]]
    admissions = _to_datetime(admissions, ["admittime"])

    merged = cohort_all.merge(admissions, on=["subject_id", "hadm_id"], how="left")
    merged = merged.sort_values(["subject_id", "admittime"])

    first_stemi_adm = merged.groupby("subject_id", as_index=False).head(1)
    cohort_final = first_stemi_adm[["subject_id", "hadm_id"]].drop_duplicates().reset_index(drop=True)

    print(f"[COHORT] 원래 STEMI 입원 후보 row 수: {len(cohort_all)}")
    print(f"[COHORT] subject_id 기준 '첫 STEMI 입원'만 남긴 row 수: {len(cohort_final)}")

    return cohort_final


def load_source_tables() -> Dict[str, pd.DataFrame]:
    """
    MIMIC-IV 원본 테이블들을 로딩.
    - Troponin은 labevents_troponin.csv만 사용.
    """
    print("[LOAD] admissions, patients, icustays...")
    admissions = pd.read_csv(os.path.join(HOSP_DIR, "admissions.csv"))
    patients = pd.read_csv(os.path.join(HOSP_DIR, "patients.csv"))
    icustays = pd.read_csv(os.path.join(ICU_DIR, "icustays.csv"))

    print("[LOAD] edstays (있으면 로딩)...")
    edstays_path = os.path.join(ED_DIR, "edstays.csv")
    edstays = pd.read_csv(edstays_path) if os.path.exists(edstays_path) else None

    print("[LOAD] labevents_troponin ...")
    labevents_trop = load_labevents_troponin()

    print("[LOAD] procedures_icd, prescriptions, ECG machine_measurements...")
    procedures_icd = pd.read_csv(os.path.join(HOSP_DIR, "procedures_icd.csv"))
    prescriptions = pd.read_csv(os.path.join(HOSP_DIR, "prescriptions.csv"))
    ecg = pd.read_csv(os.path.join(ECG_DIR, "machine_measurements.csv"))

    return {
        "admissions": admissions,
        "patients": patients,
        "icustays": icustays,
        "edstays": edstays,
        "labevents_trop": labevents_trop,
        "procedures_icd": procedures_icd,
        "prescriptions": prescriptions,
        "ecg": ecg,
    }


# ==============================
# 2-1. 디버그용 ED/ICU 매칭 상황 확인
# ==============================

def debug_check_ed_icu(cohort: pd.DataFrame, tables: Dict[str, pd.DataFrame]):
    edstays = tables["edstays"]
    icustays = tables["icustays"]

    print("\n[DEBUG] ===== ED / ICU 매칭 상태 점검 =====")

    # 1) ED
    if edstays is None:
        print("[DEBUG][ED] edstays.csv 를 찾지 못했습니다. ED 이벤트는 생성되지 않습니다.")
    else:
        print(f"[DEBUG][ED] edstays 원본 row 수: {len(edstays)}")
        print(f"[DEBUG][ED] cohort row 수: {len(cohort)}")

        ed_merge = edstays.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
        print(f"[DEBUG][ED] (subject_id, hadm_id) 기준 merge 후 row 수: {len(ed_merge)}")

        if len(ed_merge) > 0:
            print("[DEBUG][ED] 예시 5개:")
            print(ed_merge[["subject_id", "hadm_id", "intime", "outtime"]].head())
        else:
            print("[DEBUG][ED] 매칭된 ED row가 없습니다.")

    # 2) ICU
    icu = icustays
    print(f"\n[DEBUG][ICU] icustays 원본 row 수: {len(icu)}")
    icu_merge = icu.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    print(f"[DEBUG][ICU] (subject_id, hadm_id) 기준 merge 후 row 수: {len(icu_merge)}")

    if len(icu_merge) > 0:
        print("[DEBUG][ICU] 예시 5개:")
        print(icu_merge[["subject_id", "hadm_id", "intime", "outtime"]].head())
    else:
        print("[DEBUG][ICU] 매칭된 ICU row가 없습니다.")


# ==============================
# 3. Event 생성 함수들
# ==============================

def build_ed_events(cohort: pd.DataFrame, edstays: pd.DataFrame) -> pd.DataFrame:
    """
    ED_ARRIVAL, ED_DEPARTURE 이벤트 생성.
    edstays: subject_id, hadm_id, intime, outtime 가정.
    """
    if edstays is None:
        return pd.DataFrame(columns=["subject_id", "hadm_id", "event_name", "timestamp", "attributes"])

    ed = edstays.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    ed = _to_datetime(ed, ["intime", "outtime"])

    events = []
    for _, row in ed.iterrows():
        if pd.notnull(row.get("intime")):
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ED_ARRIVAL",
                "timestamp": row["intime"],
                "attributes": _attrs_to_json({}),
            })
        if pd.notnull(row.get("outtime")):
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ED_DEPARTURE",
                "timestamp": row["outtime"],
                "attributes": _attrs_to_json({}),
            })

    return pd.DataFrame(events)


def build_admission_events(cohort: pd.DataFrame,
                           admissions: pd.DataFrame,
                           patients: pd.DataFrame) -> pd.DataFrame:
    """
    DISCHARGE, DEATH 이벤트 생성.
    DISCHARGE: admissions.dischtime
    DEATH: patients.dod
    """
    adm = admissions.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    adm = _to_datetime(adm, ["admittime", "dischtime"])
    pat = _to_datetime(patients.copy(), ["dod"])

    adm = adm.merge(pat[["subject_id", "dod"]], on="subject_id", how="left")

    events = []
    for _, row in adm.iterrows():
        if pd.notnull(row.get("dischtime")):
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "DISCHARGE",
                "timestamp": row["dischtime"],
                "attributes": _attrs_to_json({}),
            })
        if pd.notnull(row.get("dod")):
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "DEATH",
                "timestamp": row["dod"],
                "attributes": _attrs_to_json({}),
            })

    return pd.DataFrame(events)


def build_icu_events(cohort: pd.DataFrame, icustays: pd.DataFrame) -> pd.DataFrame:
    """
    ICU_INTIME, ICU_OUTTIME 이벤트 생성.
    icustays: subject_id, hadm_id, intime, outtime, first_careunit, last_careunit, stay_id 가정.
    """
    icu = icustays.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    icu = _to_datetime(icu, ["intime", "outtime"])

    events = []
    for _, row in icu.iterrows():
        attrs = {
            "first_careunit": row.get("first_careunit", None),
            "last_careunit": row.get("last_careunit", None),
            "stay_id": row.get("stay_id", None),
        }
        if pd.notnull(row.get("intime")):
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ICU_INTIME",
                "timestamp": row["intime"],
                "attributes": _attrs_to_json(attrs),
            })
        if pd.notnull(row.get("outtime")):
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ICU_OUTTIME",
                "timestamp": row["outtime"],
                "attributes": _attrs_to_json(attrs),
            })

    return pd.DataFrame(events)


def build_troponin_events(cohort: pd.DataFrame,
                          labevents_trop: pd.DataFrame,
                          positive_threshold: float) -> pd.DataFrame:
    """
    TROP_TAKEN, TROP_POSITIVE 이벤트 생성.
    labevents_trop: Troponin만 필터된 labevents_troponin.csv
      (subject_id, hadm_id, itemid, charttime, valuenum 등 포함)
    """
    lab = labevents_trop.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    lab = _to_datetime(lab, ["charttime"])

    events = []

    # TROP_TAKEN
    for _, row in lab.iterrows():
        if pd.notnull(row.get("charttime")):
            attrs = {
                "itemid": row.get("itemid", None),
                "valuenum": row.get("valuenum", None),
                "value": row.get("value", None),
                "flag": row.get("flag", None),
            }
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "TROP_TAKEN",
                "timestamp": row["charttime"],
                "attributes": _attrs_to_json(attrs),
            })

    # TROP_POSITIVE: hadm_id 기준 첫 양성 시점
    if "valuenum" in lab.columns:
        lab_pos = lab[lab["valuenum"] >= positive_threshold].copy()
        lab_pos = lab_pos.dropna(subset=["charttime"])
        lab_pos_sorted = lab_pos.sort_values(["subject_id", "hadm_id", "charttime"])
        first_pos = lab_pos_sorted.groupby(["subject_id", "hadm_id"], as_index=False).head(1)

        for _, row in first_pos.iterrows():
            attrs = {
                "itemid": row.get("itemid", None),
                "valuenum": row.get("valuenum", None),
                "value": row.get("value", None),
                "flag": row.get("flag", None),
            }
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "TROP_POSITIVE",
                "timestamp": row["charttime"],
                "attributes": _attrs_to_json(attrs),
            })

    return pd.DataFrame(events)


def build_ecg_events(cohort: pd.DataFrame, ecg: pd.DataFrame) -> pd.DataFrame:
    """
    ECG_TAKEN, ECG_STEMI_FLAG 이벤트 생성

    - MIMIC-IV-ECG는 보통 hadm_id가 없고 subject_id만 있음.
    - 우리는 cohort를 '각 subject당 첫 STEMI 입원(hadm_id 하나)'로 만들었으므로,
      subject_id로만 join해서 해당 subject의 hadm_id를 붙인다.
    - 시간 컬럼은 ecg_time 또는 charttime 중 존재하는 것을 사용.
    """

    ecg_c = ecg.copy()

    # 시간 컬럼 통일: ecg_time -> charttime
    if "charttime" not in ecg_c.columns:
        if "ecg_time" in ecg_c.columns:
            ecg_c = ecg_c.rename(columns={"ecg_time": "charttime"})
        else:
            raise ValueError(
                "ECG 데이터에 charttime/ecg_time 둘 다 없습니다. "
                "시간 컬럼 이름을 확인해 주세요."
            )

    # cohort에서 subject_id당 hadm_id 하나만 남겨 join용 매핑 생성
    subj_hadm_map = cohort[["subject_id", "hadm_id"]].drop_duplicates()

    # subject_id 기준으로 join
    ecg_c = ecg_c.merge(subj_hadm_map, on="subject_id", how="inner")

    # datetime 변환
    ecg_c = _to_datetime(ecg_c, ["charttime"])

    # STEMI flag 판정 (machine_measurements + report_0~)
    def has_stemi_flag(row) -> bool:
        mm = str(row.get("machine_measurements", "")).upper()
        reports = []
        for i in range(30):
            col = f"report_{i}"
            if col in row.index:
                reports.append(str(row.get(col, "")))
        rep_text = " ".join(reports).upper()
        text = mm + " " + rep_text

        if "STEMI" in text:
            return True
        if "ST ELEVATION" in text:
            return True
        return False

    ecg_c["is_stemi"] = ecg_c.apply(has_stemi_flag, axis=1)

    # 이벤트 생성
    events = []
    for _, row in ecg_c.iterrows():
        ts = row.get("charttime")
        if pd.isna(ts):
            continue

        # ECG_TAKEN
        events.append({
            "subject_id": row["subject_id"],
            "hadm_id": row["hadm_id"],   # cohort에서 붙인 첫 STEMI 입원
            "event_name": "ECG_TAKEN",
            "timestamp": ts,
            "attributes": _attrs_to_json({}),
        })

        # ECG_STEMI_FLAG
        if row["is_stemi"]:
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ECG_STEMI_FLAG",
                "timestamp": ts,
                "attributes": _attrs_to_json({}),
            })

    return pd.DataFrame(events)


def build_pci_events(cohort: pd.DataFrame,
                     procedures_icd: pd.DataFrame,
                     pci_icd9_prefixes: List[str],
                     pci_icd10_prefixes: List[str]) -> pd.DataFrame:
    """
    PCI_START 이벤트 생성 (날짜 단위).
    procedures_icd: subject_id, hadm_id, icd_code, icd_version, chartdate 가정.
    """
    proc = procedures_icd.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    proc = _to_datetime(proc, ["chartdate"])

    def is_pci(code: str, version: float) -> bool:
        if pd.isna(code):
            return False
        code_str = str(code)
        if version == 9:
            return any(code_str.startswith(p) for p in pci_icd9_prefixes)
        elif version == 10:
            return any(code_str.startswith(p) for p in pci_icd10_prefixes)
        else:
            return False

    proc["is_pci"] = proc.apply(
        lambda r: is_pci(r.get("icd_code", None), r.get("icd_version", None)), axis=1
    )
    pci_rows = proc[proc["is_pci"]].copy()

    events = []
    for _, row in pci_rows.iterrows():
        if pd.notnull(row.get("chartdate")):
            attrs = {
                "icd_code": row.get("icd_code", None),
                "icd_version": row.get("icd_version", None),
            }
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "PCI_START",
                "timestamp": row["chartdate"],
                "attributes": _attrs_to_json(attrs),
            })

    return pd.DataFrame(events)


def build_antiplatelet_events(cohort: pd.DataFrame,
                              prescriptions: pd.DataFrame,
                              drug_name_list: List[str]) -> pd.DataFrame:
    """
    ANTI_PLT_ORDER, ANTI_PLT_ADMIN 이벤트 생성.
    prescriptions: subject_id, hadm_id, drug, starttime, stoptime,
                   route, dose_val_rx, dose_unit_rx 가정.
    """
    rx = prescriptions.merge(cohort, on=["subject_id", "hadm_id"], how="inner")
    rx = _to_datetime(rx, ["starttime", "stoptime"])

    drug_lower_list = [d.lower() for d in drug_name_list]
    rx["drug_lower"] = rx["drug"].astype(str).str.lower()
    rx = rx[rx["drug_lower"].isin(drug_lower_list)]

    events = []
    for _, row in rx.iterrows():
        if pd.notnull(row.get("starttime")):
            attrs = {
                "drug": row.get("drug", None),
                "route": row.get("route", None),
                "dose_val_rx": row.get("dose_val_rx", None),
                "dose_unit_rx": row.get("dose_unit_rx", None),
            }
            # ORDER
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ANTI_PLT_ORDER",
                "timestamp": row["starttime"],
                "attributes": _attrs_to_json(attrs),
            })
            # ADMIN (지금은 starttime과 동일 시점으로 처리)
            events.append({
                "subject_id": row["subject_id"],
                "hadm_id": row["hadm_id"],
                "event_name": "ANTI_PLT_ADMIN",
                "timestamp": row["starttime"],
                "attributes": _attrs_to_json(attrs),
            })

    return pd.DataFrame(events)


# ==============================
# 4. 후처리: DISCHARGE/DEATH 이후 제거 + ED_ARRIVAL 이후만 유지
# ==============================

def trim_after_discharge_or_death(all_events: pd.DataFrame) -> pd.DataFrame:
    """
    한 환자(hadm_id)에 대해 DISCHARGE 또는 DEATH 이후의 이벤트는 모두 제거.
    - 둘 다 있으면 더 이른 시점을 기준으로 잘라냄.
    """
    if all_events.empty:
        return all_events

    def _trim_one_stay(df: pd.DataFrame) -> pd.DataFrame:
        end_mask = df["event_name"].isin(["DISCHARGE", "DEATH"])
        if not end_mask.any():
            return df
        end_time = df.loc[end_mask, "timestamp"].min()
        return df[df["timestamp"] <= end_time]

    return (
        all_events
        .groupby("hadm_id", group_keys=False)
        .apply(_trim_one_stay)
        .reset_index(drop=True)
    )


def filter_events_after_ed(all_events: pd.DataFrame) -> pd.DataFrame:
    """
    한 환자(hadm_id)에 대해 ED_ARRIVAL 이후의 이벤트만 남긴다.
    - ED_ARRIVAL이 여러 개면 가장 이른 ED_ARRIVAL 기준으로 자른다.
    - ED_ARRIVAL이 전혀 없는 hadm_id는 통째로 제거.
    """
    if all_events.empty:
        return all_events

    def _keep_from_first_ed(df: pd.DataFrame) -> pd.DataFrame:
        mask_ed = (df["event_name"] == "ED_ARRIVAL")
        if not mask_ed.any():
            # ED 없는 입원은 제거
            return pd.DataFrame(columns=df.columns)
        start_time = df.loc[mask_ed, "timestamp"].min()
        return df[df["timestamp"] >= start_time]

    return (
        all_events
        .groupby("hadm_id", group_keys=False)
        .apply(_keep_from_first_ed)
        .reset_index(drop=True)
    )


# ==============================
# 5. Event Log 통합 함수
# ==============================

def build_event_log(cohort: pd.DataFrame, tables: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    """
    주어진 cohort subset(subject_id, hadm_id)에 대해 Event Log 생성.
    최종 컬럼:
      - case_id
      - subject_id
      - hadm_id
      - event_name
      - timestamp
      - attributes(JSON 문자열)
    """
    admissions = tables["admissions"]
    patients = tables["patients"]
    icustays = tables["icustays"]
    edstays = tables["edstays"]
    labevents_trop = tables["labevents_trop"]
    procedures_icd = tables["procedures_icd"]
    prescriptions = tables["prescriptions"]
    ecg = tables["ecg"]

    ed_events = build_ed_events(cohort, edstays)
    adm_events = build_admission_events(cohort, admissions, patients)
    icu_events = build_icu_events(cohort, icustays)
    trop_events = build_troponin_events(cohort, labevents_trop, TROP_POS_THRESHOLD)
    ecg_events = build_ecg_events(cohort, ecg)
    pci_events = build_pci_events(cohort, procedures_icd,
                                  pci_icd9_prefixes=PCI_ICD9_PREFIXES,
                                  pci_icd10_prefixes=PCI_ICD10_PREFIXES)
    antiplatelet_events = build_antiplatelet_events(cohort, prescriptions, ANTI_PLT_DRUGS)

    all_events = pd.concat(
        [
            ed_events,
            adm_events,
            icu_events,
            trop_events,
            ecg_events,
            pci_events,
            antiplatelet_events,
        ],
        ignore_index=True
    )

    if all_events.empty:
        return pd.DataFrame(columns=["case_id", "subject_id", "hadm_id", "event_name", "timestamp", "attributes"])

    # timestamp 정리
    all_events = all_events.dropna(subset=["timestamp"])
    all_events["timestamp"] = pd.to_datetime(all_events["timestamp"], errors="coerce")
    all_events = all_events.dropna(subset=["timestamp"])

    # 우선 시간/이름 기준 정렬
    all_events = all_events.sort_values(
        by=["subject_id", "hadm_id", "timestamp", "event_name"]
    ).reset_index(drop=True)

    # 1) DISCHARGE / DEATH 이후 이벤트 제거
    all_events = trim_after_discharge_or_death(all_events)

    # 2) ED_ARRIVAL 이후 이벤트만 유지 (ED가 하나라도 있을 때만 적용)
    if (all_events["event_name"] == "ED_ARRIVAL").any():
        all_events = filter_events_after_ed(all_events)
    else:
        print("[WARN] 전체 Event Log에 ED_ARRIVAL 이벤트가 없어 ED 이후 필터링을 건너뜁니다.")

    if all_events.empty:
        return pd.DataFrame(columns=["case_id", "subject_id", "hadm_id", "event_name", "timestamp", "attributes"])

    # case_id = hadm_id
    all_events["case_id"] = all_events["hadm_id"]

    # 최종 컬럼 정리
    all_events = all_events[
        ["case_id", "subject_id", "hadm_id", "event_name", "timestamp", "attributes"]
    ]

    return all_events


# ==============================
# 6. batch + versioning MAIN
# ==============================

def main():
    # 1) 코호트 로딩 (각 subject당 첫 STEMI 입원만 남김)
    cohort = load_cohort_first_stemi_admission(COHORT_PATH)
    tables = load_source_tables()

    # 디버그용 ED/ICU 매칭 상태 확인
    debug_check_ed_icu(cohort, tables)

    print(f"[INFO] 최종 STEMI cohort size (첫 입원 기준): {len(cohort)}")

    # 기존 전체 Event Log 파일이 있으면 삭제
    if os.path.exists(EVENT_LOG_FULL_PATH):
        os.remove(EVENT_LOG_FULL_PATH)
        print(f"[INFO] 기존 전체 Event Log 파일 삭제: {EVENT_LOG_FULL_PATH}")

    header_written_full = False
    version = CHUNK_VERSION_START

    n = len(cohort)
    for start in range(0, n, BATCH_SIZE):
        end = min(start + BATCH_SIZE, n)
        sub_cohort = cohort.iloc[start:end].copy()

        print(f"\n[INFO] Batch 처리중: rows {start} ~ {end-1} "
              f"(size={len(sub_cohort)}), version={version}")

        event_log_chunk = build_event_log(sub_cohort, tables)
        print(f"[INFO]   → 생성된 이벤트 row 수: {len(event_log_chunk)}")

        # (1) batch 전용 파일 저장: cohort_ver{version}.csv
        chunk_path = os.path.join(OUTPUT_DIR, f"cohort_ver{version}.csv")
        event_log_chunk.to_csv(chunk_path, index=False)
        print(f"[INFO]   → chunk 파일 저장: {chunk_path}")

        # (2) 전체 Event Log 파일에 append
        if len(event_log_chunk) > 0:
            mode = "a"
            header = not header_written_full
            event_log_chunk.to_csv(EVENT_LOG_FULL_PATH,
                                   mode=mode,
                                   header=header,
                                   index=False)
            header_written_full = True
            print(f"[INFO]   → 전체 Event Log에 append: {EVENT_LOG_FULL_PATH}")
        else:
            print("[WARN]   → 이 batch에서는 생성된 이벤트가 없습니다.")

        version = 141

    print("\n[INFO] 모든 batch 처리가 완료되었습니다.")
    print(f"[INFO] 최종 전체 Event Log 경로: {EVENT_LOG_FULL_PATH}")
    print(f"[INFO] 마지막 사용된 version 번호: {141}")


if __name__ == "__main__":
    main()


[COHORT] 원래 STEMI 입원 후보 row 수: 1929
[COHORT] subject_id 기준 '첫 STEMI 입원'만 남긴 row 수: 1878
[LOAD] admissions, patients, icustays...
[LOAD] edstays (있으면 로딩)...
[LOAD] labevents_troponin ...
[LOAD] labevents_troponin.csv 로딩: ../../data/MIMIC4-hosp-icu\labevents_troponin.csv
[LOAD] procedures_icd, prescriptions, ECG machine_measurements...


C:\Users\User\AppData\Local\Temp\ipykernel_19084\3676839491.py:142: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv(os.path.join(HOSP_DIR, "prescriptions.csv"))
C:\Users\User\AppData\Local\Temp\ipykernel_19084\3676839491.py:143: DtypeWarning: Columns (16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  ecg = pd.read_csv(os.path.join(ECG_DIR, "machine_measurements.csv"))



[DEBUG] ===== ED / ICU 매칭 상태 점검 =====
[DEBUG][ED] edstays.csv 를 찾지 못했습니다. ED 이벤트는 생성되지 않습니다.

[DEBUG][ICU] icustays 원본 row 수: 94458
[DEBUG][ICU] (subject_id, hadm_id) 기준 merge 후 row 수: 963
[DEBUG][ICU] 예시 5개:
   subject_id   hadm_id               intime              outtime
0    10010058  26359957  2147-11-18 03:19:00  2147-11-19 08:53:33
1    10012438  22764825  2178-06-07 21:57:00  2178-06-08 15:51:15
2    10013310  27682188  2153-05-06 18:28:00  2153-05-07 20:47:19
3    10015860  24698912  2192-05-12 09:31:00  2192-05-13 00:55:45
4    10055344  29209451  2171-10-31 19:38:25  2171-11-02 19:46:41
[INFO] 최종 STEMI cohort size (첫 입원 기준): 1878
[INFO] 기존 전체 Event Log 파일 삭제: ./cohort\event_log_stemi_all.csv

[INFO] Batch 처리중: rows 0 ~ 99 (size=100), version=140


C:\Users\User\AppData\Local\Temp\ipykernel_19084\3676839491.py:613: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_events = pd.concat(
C:\Users\User\AppData\Local\Temp\ipykernel_19084\3676839491.py:547: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_events


[WARN] 전체 Event Log에 ED_ARRIVAL 이벤트가 없어 ED 이후 필터링을 건너뜁니다.
[INFO]   → 생성된 이벤트 row 수: 1699
[INFO]   → chunk 파일 저장: ./cohort\cohort_ver140.csv
[INFO]   → 전체 Event Log에 append: ./cohort\event_log_stemi_all.csv

[INFO] Batch 처리중: rows 100 ~ 199 (size=100), version=141


...

[WARN] 전체 Event Log에 ED_ARRIVAL 이벤트가 없어 ED 이후 필터링을 건너뜁니다.
[INFO]   → 생성된 이벤트 row 수: 1898
[INFO]   → chunk 파일 저장: ./cohort\cohort_ver141.csv
[INFO]   → 전체 Event Log에 append: ./cohort\event_log_stemi_all.csv

[INFO] Batch 처리중: rows 1700 ~ 1799 (size=100), version=141


C:\Users\User\AppData\Local\Temp\ipykernel_19084\3676839491.py:613: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_events = pd.concat(
C:\Users\User\AppData\Local\Temp\ipykernel_19084\3676839491.py:547: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_events


[WARN] 전체 Event Log에 ED_ARRIVAL 이벤트가 없어 ED 이후 필터링을 건너뜁니다.
[INFO]   → 생성된 이벤트 row 수: 1642
[INFO]   → chunk 파일 저장: ./cohort\cohort_ver141.csv
[INFO]   → 전체 Event Log에 append: ./cohort\event_log_stemi_all.csv

[INFO] Batch 처리중: rows 1800 ~ 1877 (size=78), version=141


C:\Users\User\AppData\Local\Temp\ipykernel_19084\3676839491.py:613: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_events = pd.concat(
C:\Users\User\AppData\Local\Temp\ipykernel_19084\3676839491.py:547: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_events


[WARN] 전체 Event Log에 ED_ARRIVAL 이벤트가 없어 ED 이후 필터링을 건너뜁니다.
[INFO]   → 생성된 이벤트 row 수: 1499
[INFO]   → chunk 파일 저장: ./cohort\cohort_ver141.csv
[INFO]   → 전체 Event Log에 append: ./cohort\event_log_stemi_all.csv

[INFO] 모든 batch 처리가 완료되었습니다.
[INFO] 최종 전체 Event Log 경로: ./cohort\event_log_stemi_all.csv
[INFO] 마지막 사용된 version 번호: 141
